In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        ## Only 1 block. It takes input from 1 channel to 32 channels
        self.conv1 = nn.Conv2d(1, 8, 3) #i 28 0 26 RF 3
        self.bn1 = nn.BatchNorm2d(num_features=8, eps=1e-05, momentum=0.1)
        self.conv2 = nn.Conv2d(8, 16, 3) #i 26 o 24 RF 5
        self.bn2 = nn.BatchNorm2d(num_features=16, eps=1e-05, momentum=0.1)
        self.pool1 = nn.MaxPool2d(2, 2) #i 24 o 12 RF 10
        self.dropout = nn.Dropout(0.1)
        
        self.conv3 = nn.Conv2d(16, 32, 3) #i 12 o 10
        self.bn3 = nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1)

        ## Transition, uses 1x1 conv to reduce channel count
        ## keep under 20k params
        self.unitconv = nn.Conv2d(32, 16, 1) #i 10 o8
        self.unitbn = nn.BatchNorm2d(num_features=16, eps=1e-05, momentum=0.1)

        ## Tail block, increase channels back to 32, use GAP
        self.conv4 = nn.Conv2d(16, 32, 3) #i 8 o 06
        self.bn4 = nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1)
        self.pool2 = nn.MaxPool2d(2, 2) #i 6 o 3
        
        ## have 32 filters of 3x3 - reduce to 1x1 using GAP
        self.conv7 = nn.Conv2d(32, 10, 3) #i3 o 2
        self.gap = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        ## block 1
        x = self.conv1(x) #28->26, c=8
        x = self.bn1(x) 
        x = F.relu(x)
        x = self.conv2(x) #26->24, c=16
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool1(x) #24->12
        x = self.dropout(x)
        x = self.conv3(x) #12->10, c=32
        x = self.bn3(x)
        x = F.relu(x)

        ## Transition 1x1 conv block
        x = self.unitconv(x) #c=16
        x = self.unitbn(x) 
        x = F.relu(x)

        ## Tail block
        x = self.conv4(x) #10->8, c=32
        x = self.bn4(x)
        x = F.relu(x)
        x = self.pool2(x) #8->4
        x = self.conv7(x) #4->2, c=10
        x = F.relu(x)
        ## use GAP
        x = self.gap(x)   #1, c=10

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [16]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3           [-1, 16, 24, 24]           1,168
       BatchNorm2d-4           [-1, 16, 24, 24]              32
         MaxPool2d-5           [-1, 16, 12, 12]               0
           Dropout-6           [-1, 16, 12, 12]               0
            Conv2d-7           [-1, 32, 10, 10]           4,640
       BatchNorm2d-8           [-1, 32, 10, 10]              64
            Conv2d-9           [-1, 16, 10, 10]             528
      BatchNorm2d-10           [-1, 16, 10, 10]              32
           Conv2d-11             [-1, 32, 8, 8]           4,640
      BatchNorm2d-12             [-1, 32, 8, 8]              64
        MaxPool2d-13             [-1, 32, 4, 4]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [17]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [18]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:

model = Net().to(device)
## orig
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
## try 1
## lr = 0.001, accuracy = 98.7, 98.77, 99.12 ...
## lr = 0.0005, batchsize=128 --- accuracy = 98.86, 99.00, 99.36 ...
## lr = 0.0005, batchsize=256 --- accuracy = 98.36, 99.22, 98.63, 99.22, 99.12 xxx
## lr = 0.0001, accuracy = 98.35, 98.92, 99.08
## lr = 0.00001, accuracy = 93.15, 95.64, 96.64, 97.42, 97.83, 98.10
optimizer = optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.16702072322368622 batch_id=234: 100%|██████████| 235/235 [00:19<00:00, 12.23it/s]



Test set: Average loss: 0.1431, Accuracy: 9715/10000 (97%)



loss=0.0775439664721489 batch_id=234: 100%|██████████| 235/235 [00:19<00:00, 12.17it/s]



Test set: Average loss: 0.0776, Accuracy: 9810/10000 (98%)



loss=0.049406636506319046 batch_id=234: 100%|██████████| 235/235 [00:19<00:00, 12.28it/s]


...


Test set: Average loss: 0.0202, Accuracy: 9943/10000 (99%)



loss=0.00431703170761466 batch_id=234: 100%|██████████| 235/235 [00:19<00:00, 12.19it/s]



Test set: Average loss: 0.0195, Accuracy: 9939/10000 (99%)



loss=0.020658010616898537 batch_id=234: 100%|██████████| 235/235 [00:19<00:00, 12.11it/s]



Test set: Average loss: 0.0206, Accuracy: 9934/10000 (99%)

